In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnablePassthrough, RunnableBranch, RunnableLambda

# Load environment variables (e.g., API key)
load_dotenv()

# --- 1. Define Prompts ---

# Prompt to generate the detailed report
prompt1 = PromptTemplate(
    template="Write a detailed report on {topic}.",
    input_variables=["topic"]
)

# Prompt to summarize the report
prompt2 = PromptTemplate(
    template="Summarize the following report in a concise manner: {text}.",
    input_variables=["text"]
)

# --- 2. Define Model and Parser ---

parser = StrOutputParser()
# Assuming 'gemini-2.0-flash' is accessible; adjusting to a current available model name
# If this still causes an error, check your API key and model availability.
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

# --- 3. Define Conditional Logic and Chains ---

# This inner chain is executed when the report IS longer than 500 words.
# FIX: The RunnableLambda maps the incoming string (the report text)
# to the dictionary key 'text' expected by prompt2.
summarization_chain = RunnableSequence(
    # Map the incoming string 'text' to the dictionary {'text': text}
    RunnableLambda(lambda text: {"text": text}),
    prompt2,
    model,
    parser
)

# Define the branching logic
# FIX: Changed the second conditional pair to be the default RunnablePassThrough()
# If the first condition (> 500 words) is True, the summarization_chain runs.
# Otherwise (<= 500 words), the default RunnablePassthrough() runs.
branch = RunnableBranch(
    # True Condition: Report is too long (> 500 words) -> Summarize it
    (lambda x: len(x.split()) > 500, summarization_chain),

    # Default/Else Condition: Report is short enough (<= 500 words) -> Pass it through
    RunnablePassthrough()
)

# The main chain: (Generate Report) -> (Conditional Branch)
chain = RunnableSequence(
    RunnableSequence(prompt1, model, parser).with_config(run_name="Generate Report"),
    branch.with_config(run_name="Word Count Check & Action")
)

# --- 4. Invoke and Print ---

# Running the chain (the actual output depends on the model's response length)
print(f"--- Running Chain for Topic: LangChain ---\n")
print(chain.invoke({"topic": "LangChain"}))
print(f"\n--- End of Output ---\n")

# Print the ASCII graph for visualization
chain.get_graph().print_ascii()


--- Running Chain for Topic: LangChain ---

LangChain is an open-source framework designed to simplify the development of applications powered by Large Language Models (LLMs). It addresses the inherent limitations of raw LLMs (like limited context, lack of external knowledge, and inability to perform actions) by providing a modular and structured approach to connect them with external data sources, computational tools, and memory.

The framework is built around six core components:
1.  **Models:** Interfaces to various LLM providers.
2.  **Prompts:** Tools for constructing and managing LLM inputs.
3.  **Chains:** Sequences of operations, enabling multi-step reasoning and workflows (e.g., using LangChain Expression Language - LCEL).
4.  **Retrieval:** Methods for integrating external data (like documents from vector stores) into the LLM's context, crucial for Retrieval Augmented Generation (RAG).
5.  **Memory:** Mechanisms to persist state and conversation history across interactions.
6